In [11]:
import vk_api
import io
import json, codecs
import time

In [ ]:
"""
Загружаем данные из vk_api, сохраняем на диск
"""

In [150]:
# credentials: нужно указать логин/пароль к vk.com
# чтобы случайно не закомитить версию с логин/пароль - выносим эти данные в отдельный файл, который исключаем в .gitignore
credentialsFileName = "vk_login_password.txt" # формат файла: 1-я строка = логин, 2-я строка = пароль
# затираем значения, которые могли сохраниться с прошлых запусков (в notebook)
login = '+71231234567' 
password = ''

# загружаем значения из файла
try:
    loginFile = open(credentialsFileName, 'r')
    login = loginFile.readline().strip() # в формате: '+71231234567'
    password = loginFile.readline().strip()
except IOError:
    print ("No file with login/password: создайте файл " + credentialsFileName + ", в котором 1-я строка = логин, 2-я строка = пароль")

In [151]:
# используем:
#  wrapper-lib vk_api: https://github.com/python273/vk_api
#  описание Api от vk: https://vk.com/dev/groups.search?params[q]=bitcoin&params[future]=0&params[market]=0&params[offset]=3&params[count]=3&params[v]=5.92

In [152]:
# параметры запуска
GROUP_KEYWORD = 'bitcoin' # интересуют сообщества, в названии которых есть такое слово
COUNT_GROUPS = 50 # обработаем столько сообществ
COUNT_MESSAGES_PER_GROUP = 100 # в каждом сообществе, со стены возьмем столько сообщений

In [153]:
# получаем соединение, api
vk_session = vk_api.VkApi(login, password)
vk_session.auth(token_only=True)
api = vk_session.get_api()

In [4]:
# константы и правила, которые используем при работе с файлами
FOLDER_PREFIX = './files/' # folder, в которую складываем файлы
FILE_EXTENSION = '.txt' # расширение для json-файлов

def fileNameFunc(rawFileName):
    return FOLDER_PREFIX + rawFileName + FILE_EXTENSION

In [155]:
# Данные по сообществам - получаем по http
def obtainGroups(api, keyword, count):
    print("obtainGroups, keyword=" + keyword + ", count=" + str(count))
    results = api.groups.search(q=keyword, count=count)
    return results['items']

In [156]:
# получить N записей со стены сообщества/пользователя
def obtainWallItems(api, owner_id, count):
    # protection: проверяем входные значения
    if(count > 100): # todo: можно приделать scroll, то есть, разбить запрос на несколько, в каждом count не более 100
        raise AssertionError("count должен быть <= 100, но передали:" + str(count))

    # основная часть
    results = api.wall.get(owner_id=owner_id, filter = 'owner', count = count, offset=0)
    print("obtainWallItems, owner_id=" + str(owner_id) + ", retrieve " + str(len(results['items'])) + ", of total " + str(results['count']))
    return results['items']

# для напоминания - сохраним в комментариях альтернативный метод vk_api для получения сообщений со стены сообщества
#tools = vk_api.VkTools(vk_session)
#wall = tools.get_all('wall.get', 10, {'owner_id': -92718200, 'filter':'owner'})

In [157]:
# записываем json в файл
def writeJsonToFile(fileName, jsonData):
    with io.open(fileName, 'w', encoding='utf-8') as f:
        f.write(json.dumps(jsonData, ensure_ascii=False, sort_keys=True, indent=4))

In [158]:
# получаем все интересующие сообщества, записываем в файл
groups = obtainGroups(api, 'bitcoin', COUNT_GROUPS)
writeJsonToFile(fileNameFunc('groups'), groups)

obtainGroups, keyword=bitcoin, count=50


In [159]:
# отфильтруем сообщества - нам нужны только те, у которых есть доступ к сообщениям на стене
group_ids = []
for group in groups:
    if(group['is_closed'] == 0):
        group_ids.append(group['id'])
    else:
        print('skip group:' + str(group['id']))
print('total groups для обработки: ' + str(len(group_ids)))

skip group:16269693
skip group:16219861
total groups для обработки: 48


In [160]:
# для каждого сообщества - получаем сообщения со стены и записваем в соотв.файл
counter = 1 # счетчик, используем в названии файлов
for group_id in group_ids:
    vk_id = -1 * group_id # vk_api.wall.get по наличию знака минус - определяет, что требуется именно сообщество
    items = obtainWallItems(api=api, owner_id=vk_id, count=COUNT_MESSAGES_PER_GROUP)
    rawFileName = str(counter) + "_" + str(group_id)
    writeJsonToFile(fileNameFunc(rawFileName), items)
    counter = counter + 1
    time.sleep(0.5) # соблюдаем этикет

obtainWallItems, owner_id=-46371384retrieve 100, of total 48228
obtainWallItems, owner_id=-92718200retrieve 100, of total 10601
obtainWallItems, owner_id=-632045retrieve 100, of total 28475
obtainWallItems, owner_id=-93604475retrieve 100, of total 14366
obtainWallItems, owner_id=-59454515retrieve 100, of total 31644
obtainWallItems, owner_id=-79363873retrieve 100, of total 830
obtainWallItems, owner_id=-51789870retrieve 100, of total 13932
obtainWallItems, owner_id=-92357206retrieve 100, of total 132
obtainWallItems, owner_id=-101271420retrieve 100, of total 5135
obtainWallItems, owner_id=-29387592retrieve 100, of total 1750
obtainWallItems, owner_id=-158943684retrieve 100, of total 149
obtainWallItems, owner_id=-122422421retrieve 100, of total 6558
obtainWallItems, owner_id=-74106120retrieve 1, of total 1
obtainWallItems, owner_id=-50725681retrieve 100, of total 1397
obtainWallItems, owner_id=-140465118retrieve 33, of total 33
obtainWallItems, owner_id=-50061159retrieve 59, of total 5

In [ ]:
"""
Анализируем сохраненные данные
"""

In [ ]:
# пройти по всем файлам, которые с названием [N]_[id].txt
# 1. Определяем набор полей, которые интересуют
#   1.1 собираем набор существующих полей -> просматриваем, выбираем те, которые интересуют
#   1.2 по выбранным полям - составляем статистику, уточняем, что именно будем выгружать
# 2. Составляем csv-файл со всеми сообщениями, сохраняем

In [18]:
from os import listdir
from os.path import isfile, join
import re
#import unicodecsv as csv
import csv

In [6]:
# функции и константы

SCRAPPED_DATA_FILE_REGEXP = re.compile(r'^([\d]+)_([\d]+).*$')

# проанализировать массив json - какие поля встречаются
def fieldsIn(fileContent):
    result = []
    for message in fileContent:
        for key, value in message.items():
            if(key not in result):
                result.append(key)
    return result

# объединяем списки, оставляем уникальные элементы
def clueFiledsFunc(acc, addFields):
    for field in addFields:
        if(field not in acc):
            acc.append(field)
    return acc

# определяем, какие поля встречаются в загруженных данных
def revealFieldNames(fileNameList):
    fields = []
    for fileName in scrapped_data_files:
        with io.open(join(FOLDER_PREFIX, fileName), 'r', encoding='utf-8') as f:
            fileContent = json.load(f)
            fields = clueFiledsFunc(fields, fieldsIn(fileContent))
    return fields

In [9]:
# названия файлов со scrapped данными
scrapped_all_files = [f for f in listdir(FOLDER_PREFIX) if isfile(join(FOLDER_PREFIX, f))]
# только те файлы, которые про сообщения на стене сообщества
scrapped_data_files = []
regexp = SCRAPPED_DATA_FILE_REGEXP
for fileName in scrapped_all_files:
    if(regexp.search(fileName)): 
        scrapped_data_files.append(fileName)

In [12]:
fields = revealFieldNames(scrapped_data_files)
print(fields)

['attachments', 'comments', 'date', 'from_id', 'id', 'is_favorite', 'likes', 'marked_as_ads', 'owner_id', 'post_source', 'post_type', 'reposts', 'text', 'views', 'is_pinned', 'signer_id', 'copy_history']


In [29]:
# поля, которые интересуют
actual_fields = {
    'comments_count': 'comments.count',
    'date':'date',
    'is_favorite':'is_favorite',
    'is_pinned':'is_pinned',
    'likes_can_like':'likes.can_like',
    'likes_can_publish':'likes.can_publish',
    'likes_count':'likes.count',
    'marked_as_ads':'marked_as_ads',
    'post_source':'post_source.type',
    'post_type':'post_type',
    'reposts_count':'reposts.count',
    'text':'text',
    'views_count':'views.count'
}
# в обработанном виде
csv_fields = []
json_fields = []
for key, value in actual_fields.items():
    csv_fields.append(key)
    json_fields.append(value)

In [30]:
# получаем значение поля в json
def revealFieldValue(jsonData,fieldPath):
    none_value = ''
    path = fieldPath.split('.')
    data = jsonData
    for step in path[:-1]:
        if step in data:
            data = data[step]
        else:
            return none_value
    data = data.get(path[-1], none_value)
    #print(data)
    return data

In [54]:
# из объекта - получаем строку для csv
def produceLine(jsonData, fieldsList):
    result = []
    for field in fieldsList:
        data = revealFieldValue(jsonData, field)
        result.append(data)
    return result

# из файла со scrapped_data получаем строки для csv
def produceCsvRowsForFile(fileName, fieldsList):
    group_id = SCRAPPED_DATA_FILE_REGEXP.match(fileName).group(2)
    result = []
    with io.open(join(FOLDER_PREFIX, fileName), 'r', encoding='utf-8') as f:
        fileContent = json.load(f)
        for doc in fileContent:
            row = [group_id] + produceLine(doc, fieldsList)
            #row.append(group_id)
            result.append(row)
    return result
            
# для всех перечисленных файлов - записываем получаемые строки в csv (в памяти держим строки только для одного файла)
def writeCsvLinesByFiles(csvWriter, fileNameList, fieldsList):
    for fileName in fileNameList:
        print('fileName: ' + fileName)
        rows = produceCsvRowsForFile(fileName, fieldsList)
        #print("***")
        #print(rows[1])
        for row in rows:            
            csvWriter.writerow(row)
            

        

In [56]:
with open('result.csv', 'w', newline='', encoding='utf-8') as csvFile:
    csvWriter = csv.writer(csvFile, delimiter=',')
    csvWriter.writerow(['group_id'] + csv_fields) # 1-я строка - это названия столбцов
    print(json_fields)
    writeCsvLinesByFiles(csvWriter, scrapped_data_files, json_fields)

['comments.count', 'date', 'is_favorite', 'is_pinned', 'likes.can_like', 'likes.can_publish', 'likes.count', 'marked_as_ads', 'post_source.type', 'post_type', 'reposts.count', 'text', 'views.count']
fileName: 10_29387592.txt
fileName: 11_158943684.txt
fileName: 12_122422421.txt
fileName: 13_74106120.txt
fileName: 14_50725681.txt
fileName: 15_140465118.txt
fileName: 16_50061159.txt
fileName: 17_50774319.txt
fileName: 18_147911279.txt
fileName: 19_62032126.txt
fileName: 1_46371384.txt
fileName: 20_30451584.txt
fileName: 21_63589724.txt
fileName: 22_19354230.txt
fileName: 23_65314134.txt
fileName: 24_33135732.txt
fileName: 25_79858655.txt
fileName: 26_70023397.txt
fileName: 27_43011371.txt
fileName: 28_113755164.txt
fileName: 29_66153827.txt
fileName: 2_92718200.txt
fileName: 30_55500226.txt
fileName: 31_61469977.txt
fileName: 32_150541976.txt
fileName: 33_53986544.txt
fileName: 34_152549864.txt
fileName: 35_83377401.txt
fileName: 36_144516691.txt
fileName: 37_59683167.txt
fileName: 38_11

In [45]:
import csv
with open('eggs.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
    spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])